In [10]:
import glob
import re
import pandas as pd
from yattag import Doc
import unicodedata

In [23]:
doc, tag, text = Doc().tagtext()
txt_file = glob.glob('AmostrasDODF/**/*.txt')

all_contracts = []

for txt in txt_file:
    instance = re.findall(r'[\s\S]+', 
                unicodedata.normalize('NFKD', open(txt, encoding='utf-8').read()))

    instance = instance[0].split('xxbcet SEÇÃO III xxecet')
    all_contracts.append(txt + '\n' + instance[-1])


secao_3 = []

for contract in all_contracts:
    secao_3.append(contract.split('\n'))


### DFA

- l = lista com toda a seção 3 de um único dodf 
- i = posição atual da lista l -> inicia-se com 0
- l_acts = lista vazia que será preenchida com os atos
- j = posição atual da lista l_acts -> inicia-se com 0
- regex = regex do ato que está sendo procurado

In [44]:
def q0(l, i, l_acts, j):
    """Estado que procura por um aviso de abertura"""

    regex5 = r'xxbcet AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+JULGAMENTO|AVISO\s+D[EO]\s+JULGAMENTO'
    regex6 = r'xxbcet AVISO\s+D[EO]\s+LICITAÇÃO'
    regex7 = r'xxbcet AVISO\s+D[EO]\s+JULGAMENTO\s+D[EO]\s+HABILITAÇÃO'
    regex8 = r'xxbcet AVISO\s+D[EO]\s+RESULTADO\s+D[EO]\s+RECURSO\s+E\s+JULGAMENTO'

    if i == len(l):
        return

    elif re.match(regex5, l[i]):
        l_acts.append('[5] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex5)

    elif re.match(regex6, l[i]):
        l_acts.append('[6] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex6)

    elif re.match(regex7, l[i]):
        l_acts.append('[7] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex7)

    elif re.match(regex8, l[i]):
        l_acts.append('[8] ' + l[0] + l[i])
        q1(l, i + 1, l_acts, j, regex8)

    else:
        q0(l, i + 1, l_acts, j)


def q1(l, i, l_acts, j, regex):
    """Estado que procura a continuação do aviso de abertura"""

    if i == len(l)-1:
        return

    regex_s = r'xxbcet [AVISOS?|EXTRATOS?|RESULTADOS?|SECRETARIA|PREGÃO]'

    if not re.match(regex, l[i]) and re.match(regex_s, l[i]) and ('xxbob' in l[i-1] or('—' in l[i-1] and 'xxbob' in l[i-1])):
        q0(l, i, l_acts, j + 1)

    elif re.match(regex, l[i]):
        q0(l, i, l_acts, j + 1)

    else:
        l_acts[j] += '\n' + l[i]
        q1(l, i + 1, l_acts, j, regex)



In [45]:
acts = []

for idx in secao_3:
    l_acts = []
    q0(idx, 0, l_acts, 0)
    acts.extend(l_acts)


In [46]:
len(acts)

73

In [47]:
acts

['[6] AmostrasDODF\\DODF 008 11-01-2012 SECAO3_txt\\004.txtxxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃO ELETRÔNICO No 8/2012. xxecet\nACOMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL torna público\nque realizará o Pregão Eletrônico PE-008/2012-CAESB, processo no 092.011281/2011, Aquisi-\nção de Reagentes Químicos e Vidrarias, conforme especificações técnicas constantes no Anexo\nI(Pedido de Aquisição/Termo de Referência), por preço equalizado, por lote cotado. Data final\nxxeob\n\nxxbob\npara Recebimento das Propostas: 26 de janeiro de 2012, às 08h30. Início da Sessão de Disputa:\n26 de janeiro 2012, às 10h00. O edital e seus anexos poderão ser encontrados no site www.\nlicitacoes-e.com.br a partir do dia 12 de janeiro de 2012. Outras informações poderão ser obtidas\npor meio do telefone (61) 3213-7275, email pregao(dcaesb.df.gov.br.\nxxbcet Brasília/DF, 10 de janeiro de 2012. xxecet\nxxbcet RAULINDO JUNIOR NAVES REZENDE xxecet\nxxbcet Pregoeiro Caes